In [2]:
import os
import math


In [4]:

def read_tsp(file_path):
    """Parses a .tsp file and extracts points as a list of (id, x, y)."""
    # Ensure the file path is correct relative to this script's directory
    # full_path = os.path.join(os.path.dirname(__file__), "..", file_path)
    points = []

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Debugging: Print all lines for verification
        # print("DEBUG: File lines:\n", lines)

        # Skip the first 5 lines and process the coordinate section
        for line in lines[5:]:
            line = line.strip()

            if line == "EOF":  # Stop parsing at "EOF"
                break

            parts = line.split()
            if len(parts) == 3:  # Ensure it has id, x, y
                node_id, x, y = int(parts[0]), float(parts[1]), float(parts[2])
                points.append((node_id, x, y))
    except FileNotFoundError:
        raise FileNotFoundError(f"Input file not found: {full_path}")
    except Exception as e:
        raise RuntimeError(f"Error reading TSP file {full_path}: {e}")

    # Debugging: Print the parsed points for verification
    # print(f"DEBUG: Parsed points: {points}")

    return points

In [5]:


def create_distance_matrix(points):
    """Creates a distance matrix from points."""
    n = len(points)
    matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            dist = euclidean_distance(points[i], points[j])
            matrix[i][j] = matrix[j][i] = dist
    return matrix


def euclidean_distance(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return math.sqrt((p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2)

In [10]:
points = read_tsp(file_path="../input/Atlanta.tsp")

In [38]:
from itertools import permutations
import time
from utils import create_distance_matrix

def brute_force_tsp(points, cutoff):
    """
    Finds the TSP tour using brute force within the time cutoff.
    :param points: List of (id, x, y) tuples.
    :param cutoff: Time limit in seconds.
    :return: (cost, tour) where cost is the total distance and tour is the list of indices.
    """
    
    start_time = time.time()
    
    # Validate input
    if not points or len(points) == 0:
        raise ValueError("No points provided for TSP")

    best_cost = float('inf')
    best_tour = []
    n = len(points)
    distance_matrix = create_distance_matrix(points)

    # cutoff_time = float(cutoff)
    # Iterate through all permutations
    for perm in permutations(range(n)):
        if time.time() - start_time > cutoff:
            break
        cost = sum(distance_matrix[perm[i]][perm[i + 1]] for i in range(n - 1))
        cost += distance_matrix[perm[-1]][perm[0]]  # Complete the cycle
        if cost < best_cost:
            best_cost = cost
            best_tour = perm

    return best_cost, list(best_tour)

In [ ]:
cost, tour = brute_force_tsp(points, cutoff=10)


In [35]:

print(f"Cost: {cost}, Tour: {tour}")

Cost: 3816909.110191864, Tour: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 18, 14, 13, 17, 10, 19, 11, 12, 16]


In [ ]:
# mst_approximation.py
from scipy.sparse.csgraph import minimum_spanning_tree

def mst_approximation_tsp(points):
    """Approximates TSP using a Minimum Spanning Tree."""
    distance_matrix = create_distance_matrix(points)
    mst = minimum_spanning_tree(distance_matrix).toarray()
    visited = set()

    # Perform a preorder traversal of the MST to get the tour
    def dfs(node, tour):
        visited.add(node)
        tour.append(node)
        for neighbor in range(len(points)):
            if mst[node][neighbor] > 0 and neighbor not in visited:
                dfs(neighbor, tour)

    tour = []
    dfs(0, tour)
    tour.append(0)  # Complete the cycle

    cost = sum(distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    return cost, tour

In [27]:
def test_mst_approximation(file_path):
    # file_path = "input/Atlanta.tsp"
    points = read_tsp(file_path)
    cost, tour = mst_approximation_tsp(points)
    print(f"Approximate TSP cost: {cost}")
    
    print(f"Approximate TSP tour: {tour}")
    return cost, tour

In [31]:
test_mst_approximation(
    file_path="input/Atlanta.tsp")

Approximate TSP cost: 305974.4741402309
Approximate TSP tour: [0, 1, 2, 16, 6, 0]


(305974.4741402309, [0, 1, 2, 16, 6, 0])

In [32]:

import random
import math
from utils import create_distance_matrix
import time


def calculate_cost(tour, distance_matrix):
    """
    Calculates the total cost of a TSP tour.
    :param tour: List of indices representing the tour.
    :param distance_matrix: 2D list or numpy array with pairwise distances.
    :return: Total cost of the tour.
    """
    cost = sum(distance_matrix[tour[i]][tour[i + 1]]
               for i in range(len(tour) - 1))
    cost += distance_matrix[tour[-1]][tour[0]]  # Return to the starting city
    return cost


def simulated_annealing_tsp(points, cutoff, seed=None):
    """Finds a near-optimal TSP tour using simulated annealing."""
    random.seed(seed)
    start_time = time.time()
    n = len(points)
    if n == 0:
        raise ValueError("No points provided for TSP")

    distance_matrix = create_distance_matrix(points)

    # Initial random tour
    current_tour = list(range(n))
    random.shuffle(current_tour)
    current_cost = calculate_cost(current_tour, distance_matrix)

    best_tour, best_cost = current_tour[:], current_cost
    T = 1000  # Initial temperature
    alpha = 0.995  # Cooling rate
    epsilon = 1e-3  # Minimum temperature

    while time.time() - start_time < cutoff and T > epsilon:
        # Swap two cities
        i, j = random.sample(range(n), 2)
        new_tour = current_tour[:]
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
        new_cost = calculate_cost(new_tour, distance_matrix)

        # Accept the new tour based on probability
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / T):
            current_tour, current_cost = new_tour, new_cost

        # Update the best solution
        if current_cost < best_cost:
            best_tour, best_cost = current_tour[:], current_cost

        T *= alpha  # Cool down

    return best_cost, best_tour


def simulated_annealing_tsp(points, cutoff, seed=None):
    """
    Solves the TSP using Simulated Annealing.
    :param points: List of (id, x, y) tuples.
    :param cutoff: Time limit in seconds.
    :param seed: Random seed for reproducibility.
    :return: (cost, tour) where cost is the total distance and tour is the list of indices.
    """
    random.seed(seed)
    n = len(points)
    if n == 0:
        raise ValueError("No points provided for TSP")

    distance_matrix = create_distance_matrix(points)

    # Step 1: Initialize with a random tour
    current_tour = list(range(n))
    random.shuffle(current_tour)
    current_cost = calculate_cost(current_tour, distance_matrix)

    best_tour = current_tour[:]
    best_cost = current_cost

    # Step 2: Define the annealing parameters
    T = 1000  # Initial temperature
    alpha = 0.995  # Cooling rate
    epsilon = 1e-3  # Minimum temperature
    start_time = time.time()

    while T > epsilon and (time.time() - start_time) < cutoff:
        # Step 3: Generate a neighbor (swap two cities)
        i, j = random.sample(range(n), 2)
        new_tour = current_tour[:]
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]

        new_cost = calculate_cost(new_tour, distance_matrix)

        # Step 4: Accept the new tour based on the acceptance probability
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / T):
            current_tour = new_tour
            current_cost = new_cost

            # Update the best solution if the new one is better
            if new_cost < best_cost:
                best_tour = new_tour
                best_cost = new_cost

        # Step 5: Cool down the temperature
        T *= alpha

    return best_cost, best_tour

In [33]:
simulated_annealing_tsp(points, cutoff=10)

(2302553.099562435,
 [4, 12, 6, 0, 1, 2, 16, 5, 9, 15, 3, 18, 14, 7, 13, 17, 8, 11, 10, 19])

In [36]:
from algorithms import brute_force, mst_approximation, simulated_annealing

In [ ]:
cost, tour = brute_force.brute_force_tsp(points, cutoff=10)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/rainylty/opt/anaconda3/envs/city8/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_41205/408185690.py", line 1, in <module>
    cost, tour = brute_force.brute_force_tsp(points, cutoff=10)
  File "/Users/rainylty/STUDY/fall24/4-Course/2-ALGxCSE6140/3-Project/CSE6140-Final-Project/code/algorithms/brute_force.py", line 28, in brute_force_tsp
  File "/Users/rainylty/STUDY/fall24/4-Course/2-ALGxCSE6140/3-Project/CSE6140-Final-Project/code/algorithms/brute_force.py", line 28, in <genexpr>
TypeError: 'int' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rainylty/opt/anaconda3/envs/city8/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = se

In [39]:

cost, tour = brute_force_tsp(points, cutoff=10)
